In [2]:
import pandas as pd
visitors = pd.read_csv('tourists.csv')
gtrends = pd.read_csv('gtrends.csv')

In [4]:
gtrends.head()

,Month,Flights to Hong Kong
0,2007-01,14
1,2007-02,14
2,2007-03,26
3,2007-04,26
4,2007-05,12


In [6]:
# predictors
visitors['Visitors_lag_1'] = visitors['Visitors'].shift(1) # autoregressive lag 1M
visitors['Visitors_lag_12'] = visitors['Visitors'].shift(12) # seasonal adjustment
visitors.head()

,Time,Visitors,Visitors_lag_1,Visitors_lag_12
0,1/1/2007,93897,NaN,NaN
1,2/1/2007,68538,93897.0,NaN
2,3/1/2007,115014,68538.0,NaN
3,4/1/2007,111047,115014.0,NaN
4,5/1/2007,96659,111047.0,NaN


In [7]:
df = pd.concat([visitors, gtrends['Flights to Hong Kong'].shift(1)], axis=1)

In [8]:
df

,Time,Visitors,Visitors_lag_1,Visitors_lag_12,Flights to Hong Kong
0,1/1/2007,93897,NaN,NaN,NaN
1,2/1/2007,68538,93897.0,NaN,14.0
2,3/1/2007,115014,68538.0,NaN,14.0
3,4/1/2007,111047,115014.0,NaN,26.0
4,5/1/2007,96659,111047.0,NaN,26.0
...,...,...,...,...,...
126,7/1/2017,98731,105460.0,93153.0,93.0
127,8/1/2017,82604,98731.0,81982.0,67.0
128,9/1/2017,85766,82604.0,93313.0,74.0
129,10/1/2017,109357,85766.0,115961.0,75.0


In [14]:
y = df['Visitors'][12:]
x = df.drop(['Time', 'Visitors'], axis=1)[12:]

In [15]:
y

12     104025
13      75997
14     118278
15     112075
16      95670
        ...  
126     98731
127     82604
128     85766
129    109357
130    121179
Name: Visitors, Length: 119, dtype: int64

In [16]:
# train set: keep data from 2007 - 2016
x_t = x[:-11]
y_t = y[:-11]

# validation set: 2017
x_v = x[-11:]
y_v = y[-11:]

In [18]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(x_t, y_t)
model.coef_.tolist()

[0.12745533775184467, 0.7870243816440682, 71.68093328258574]

In [21]:
model.intercept_

4508.535731184296

Y(t) = 4508.535731184296 + Y(t-1)*0.12745533775184467 + Y(t-12)*0.7870243816440682 + GT*71.68093328258574

In [22]:
y_t_pred = model.predict(x_t)

In [23]:
from sklearn.metrics import r2_score
r2_score(y_t, y_t_pred)

0.6626057152113144